In [1]:
import os
import sys

# 获取当前工作目录
current_dir = os.getcwd()
# 设置项目根目录
project_root = os.path.abspath(os.path.join(current_dir, ".."))
# 将项目根目录添加到sys.path
sys.path.insert(0, project_root)

In [3]:
from langchain_community.graphs import Neo4jGraph


neo4j_graph = Neo4jGraph(
    url='neo4j://192.168.0.103:7687', username='neo4j', password='password')


In [11]:
from openai import OpenAI
import os
from utils import prompt_template
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter

deepseek_key = os.environ.get('DEEPSEEK_KEY')

llm=ChatOpenAI(model_name="deepseek-chat", api_key=deepseek_key, base_url="https://api.deepseek.com")
llm_transformer = LLMGraphTransformer(llm=llm)

raw_documents = """
通知MIB定义在zxcomc-snmp-notificationirp-mib-v1.0.txt文件中，该MIB文件在alarmIRP部分新增notificationIRP对象，该MIB库主要定义统一北向接口中和通知属性相关的对象，功能如下：

ntsAttribute{ .1.3.6.1.4.1.3902.4101.10.1}分支，其下属性为通知中各字段的属性标识。

ntsNotifications{.1.3.6.1.4.1.3902.4101.10.2}分支，其下属性用于定义通知Trap标识，目前有通知上报Trap(ntsNotificationNew)。

ntsNotificationFilter{.1.3.6.1.4.1.3902.4101.10.3}分支，用于通知Trap消息的上报过滤条件，只有满足条件的通知消息才被发送到上级。
"""
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.create_documents([raw_documents])
print(documents)

In [12]:
# 提取图数据
graph_documents = llm_transformer.convert_to_graph_documents(documents)

print(graph_documents)

# 存储到 neo4j
neo4j_graph.add_graph_documents(
  graph_documents, 
  baseEntityLabel=True, 
  include_source=True
)

In [13]:
import os

from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document
deepseek_key = os.environ.get('DEEPSEEK_KEY')

llm = ChatOpenAI(temperature=0, model_name="deepseek-chat", api_key=deepseek_key, base_url="https://api.deepseek.com")

llm_transformer = LLMGraphTransformer(llm=llm)


text = """
Marie Curie, born in 1867, was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity.
She was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields.
Her husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.
She was, in 1906, the first woman to become a professor at the University of Paris.
"""
documents = [Document(page_content=text)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")